In [25]:
%run run_voting_classifier.py x33t677d prw7kpf2 prw7kpf2 AttritionFlag unclassified -p ~/Documents/Classifiers/AthenaHealth/ -s

Loading Testing & Training documents...
No classifier found in /Users/kwang/Documents/Classifiers/AthenaHealth/.
Training classifier...
Testing classifier...
Test Accuracy:96.30%


In [52]:
from luminoso_api import LuminosoClient
import json

In [29]:
client = LuminosoClient.connect('/')

In [31]:
client = client.change_path('/projects/d53m338v/prmrg4vj')

In [32]:
docs = client.get('docs', limit=25000)

In [57]:
docs_to_delete = []
length = len('NPS Score: ')
for i in range(0, len(docs)):
    for j in range(0, len(docs[i]['subsets'])):
        if 'NPS Score: ' in docs[i]['subsets'][j] and 'NULL' not in docs[i]['subsets'][j]:
            #print(docs[i]['subsets'][j][length])
            if int(docs[i]['subsets'][j][length]) >= 7:
                docs_to_delete.append(docs[i]['_id'])
        if 'NULL' in docs[i]['subsets'][j]:
            docs_to_delete.append(docs[i]['_id'])

In [58]:
doc_ids = json.dumps(docs_to_delete)
client.delete('/docs', ids=doc_ids)

ConnectionError: ('Connection aborted.', BadStatusLine('<html>\r\n',))

In [60]:
doc_ids

'["uuid-01d6c92310aa4c4a982da4a2b22df37f", "uuid-01d6c92310aa4c4a982da4a2b22df37f", "uuid-6c9eb1e7d9c34772a3be3af36e2c5e67", "uuid-302d99badaba40abb37dcab17c3f4885", "uuid-e61d83213fab4534a0bd3a01f5e1e12b", "uuid-e61d83213fab4534a0bd3a01f5e1e12b", "uuid-e3229782b4704a8bbd9bc0244167d609", "uuid-e3229782b4704a8bbd9bc0244167d609", "uuid-74f0a45d657b412fb8a8b0358475539f", "uuid-8c7cacd7b5754830bbe8e27a24e35f6c", "uuid-9c1ca32491c94e56983a71d94b4062a0", "uuid-3399a68d747145aeb15a8f829de7aec1", "uuid-3451a7d643e9423e84f6a794fbc77a2c", "uuid-3451a7d643e9423e84f6a794fbc77a2c", "uuid-0293fe04e4c1470ea5baf288fa15baec", "uuid-5554203e299b4014ae5b849d4ae3ef21", "uuid-d51f83aadb594b2b96e2b2ca6eed6135", "uuid-eb94dee48ec4425aaedc9f3decfdc7f4", "uuid-eb94dee48ec4425aaedc9f3decfdc7f4", "uuid-a9993085f3c845cf9729cbb51a2f5427", "uuid-a9993085f3c845cf9729cbb51a2f5427", "uuid-2f115adfd71f416d96810ddc67eca6b7", "uuid-2f115adfd71f416d96810ddc67eca6b7", "uuid-aca64a9c4c1547c9af324e1bab7f75f4", "uuid-5d4fd8df

In [49]:
x = [1, 2, 3]
y = [2, 4, 6]

In [54]:
linregress(y, x)[0]

0.5

In [51]:
from scipy.stats import linregress